In [ ]:
%matplotlib inline
from __future__ import division, print_function
import shutil
from io import StringIO
import numpy as np
import os 
import glob
import matplotlib.pyplot as plt
import pytraj as pt
import mdtraj as md
import itertools as it

In [ ]:
molcas_input = """*DFT energy and charges for scn
*
>>> EXPORT MOLCAS_MEM=3750
&GATEWAY
  Title = scn water rf dft
  Coord = %s
  Basis = ANO-L-VDZP
  Group = C1

  &SEWARD; CHOLesky
  &SCF
    KSDFT = M06
    charge = -1

  &LOPROP
    NoField
    MPPRop = 1
    Bond = 0.5"""

In [ ]:
cluster_script="""#!/bin/bash
# requesting the number of cores needed on exclusive nodes
#SBATCH -n 1
#SBATCH --mem-per-cpu=5000
#
# job time, change for what your job requires 
#SBATCH -t 10:00:00
# 
# job name
#SBATCH -J dft%s
#
# filenames stdout and stderr
#SBATCH -o molcas.out
#SBATCH -e molcas.err

export MOLCAS=/lunarc/nobackup/users/gtesei00/molcas-v8.1.160723-0800
export PATH=/home/gtesei00/.Molcas:/lunarc/nobackup/users/gtesei00/Molcas:$PATH
export MOLCAS_CPUS=1
export Project=dft

# change to local disk and start the mpi executable
cd $SLURM_SUBMIT_DIR
molcas $Project.input -f"""

In [ ]:
WORKDIR = '/lunarc/nobackup/users/gtesei00/dft/m06_iv'
os.chdir(WORKDIR)
for snapshot in glob.glob(WORKDIR+'/snapshots/shell_8*.xyz'):
    name = snapshot.split('/')[8].split('.')[0].split('shell')[1]
    print(name)
    c_script = cluster_script % name
    m_input = molcas_input % snapshot
    os.makedirs(name)
    
    f = open(snapshot,'r')
    filedata = f.readlines()[2:5]
    f.close()
    newdata = '3\nAngstrom\n'+''.join(filedata)
    f = open(snapshot,'w')
    f.write(newdata)
    f.close()
    
    os.chdir(name)
    with open('mol.sh', 'w') as text_file:
        text_file.write(c_script)
    with open('dft.input', 'w') as text_file:
        text_file.write(m_input)
    !sbatch mol.sh
    os.chdir(WORKDIR)

In [ ]:
import re

def find_charge(name,element='S'):
    with open(name) as origin_file:
        cnt = 4
        value = 1e4
        for line in origin_file:
            cnt += 1
            domain = re.findall(r'ATOMIC DOMAIN: '+element, line)
            if domain:
                cnt = 0                
            if cnt == 4:
                charge = re.findall(r'Total charge', line)
                if charge:
                    value = line.split(':')[1].split('\n')[0]
        return float(value)
    
WORKDIR = '/lunarc/nobackup/users/gtesei00/dft/m06_iv'
for i in [8]:
    sulfur = np.empty(0)
    carbon = np.empty(0)
    nitrogen = np.empty(0)
    for snapshot in glob.glob(WORKDIR+'/snapshots/shell_'+str(i)+'_*.xyz'):
        name = WORKDIR+'/'+snapshot.split('/')[8].split('.')[0].split('shell')[1]+'/dft.log'
        sulfur = np.append(sulfur,find_charge(name,element='S'))
        carbon = np.append(carbon,find_charge(name,element='C'))
        nitrogen = np.append(nitrogen,find_charge(name,element='N'))
    print(i,sulfur.mean(),sulfur.std())
    print(i,carbon.mean(),carbon.std())
    print(i,nitrogen.mean(),nitrogen.std())  
    print(sulfur, sulfur.mean()+carbon.mean()+nitrogen.mean())
    x = range(0,3)
    labels = ['S','C','N']
    ratio = sulfur.mean()/nitrogen.mean()
    ratio_err = ratio*np.sqrt((sulfur.std()/sulfur.mean())**2+(nitrogen.std()/nitrogen.mean())**2)
    print('ratio S/N',ratio,'+/-',ratio_err)
    y = [sulfur.mean(), carbon.mean(), nitrogen.mean()]
    yerr = [sulfur.std(), carbon.std(), nitrogen.std()]
    plt.errorbar(x,y,yerr,lw=0,marker='o',markeredgecolor=None,alpha=0.8,markersize=8,
                 elinewidth=2,capsize=5,capthick=2,label='M06/DFT, '+str(i)+'HOH')
plt.ylabel('Local Charge Number')
plt.xlabel('Atom')
plt.xticks(range(0,3),labels)
plt.xlim(-1,3)
plt.legend()
plt.show()